*This notebook serves as a rough skeleton for new example notebook additions, giving guidance on standards and best practices.  It does not need to be adhered to exactly.  The actual content of the notebook should determine the appropriate way to present that information, so deviate as needed.  Indeed, there are several example notebooks that couldn't follow this template because they do not use some components of SageMaker's functionality (e.g. [SageMaker and Redshift](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/working_with_redshift_data/working_with_redshift_data.ipynb), [TensorFlow BYOM](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb), etc.).  However, there are several elements which are mandatory for inclusion.  These are marked explicitly below.*

*NOTE: There are several best practices included in this notebook (Amazon copyright included in the notebook metadata, notebook opens in a SageMaker env (conda_python3 should be the default)).  So, even if the content is expected to deviate drammatically, starting from this notebook will help you conform to standards.*

# Title
_**Subtitle**_

---

---


## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
1. [Extensions](#Extensions)

---

## Background

_This section should contain several paragraphs describing the topic at a high level, data source, algorithms, and/or any specific libraries or engineering used._

---

## Setup

_This notebook was created and tested on an ml.< instance type > notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the notebook instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace `sagemaker.get_execution_role()` with the appropriate full IAM role arn string(s).

In [1]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
base = 'DEMO-<notebook_name_here>' # notebook auther to input a short descriptive name
prefix = 'sagemaker/' + base

role = sagemaker.get_execution_role()

Now we'll import the Python libraries we'll need.

In [2]:
#import sagemaker
#import boto3
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

---

## Data

*This section should describe the dataset at a high level.  Typically the dataset will be downloaded with a `wget` or taken from S3.  If so, make sure to have the dataset blessed ([example ticket](https://tt.amazon.com/0132482232)).  Often times this requires we properly attribute the dataset source.*

*Then we should open and explore the data briefly or deeply depending on the depth of understanding the customer will need to contextualize the remainder of the example notebook.*

*Finally, if using SageMaker built-in algorithms from a protobuf dataset, please ensure conversion is in the appropriate format (and ideally uses the SageMaker Python SDK).  Also ensure this final dataset is uploaded to the appropriate bucket + prefix S3 location.*

---

## Train

*This section mainly includes the transactional aspects of training your algorithm with SageMaker.  All notes below may be handled to a greater or lesser extent depending on whether training is done using boto3 and the AWS SDK, a generic estimator from the Python SDK, or a custom estimator from the Python SDK.*

*Regardless, the algorithm should train dynamically using the right container by region (in all regions), along the lines of:*

In [ ]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, 
                                                            'factorization-machines', 
                                                            'latest')

*It should train on the appropriate instance type with a bias toward ml.m4.xlarge (since that's free tier eligible), and should have a reasonable value for MaxRuntimeInSeconds.*

*Both input data and output locations should be set to the appropriate S3 bucket + prefix.*

*Descriptions of hyperparameters, their usage (e.g. increasing this hyperparameter will lead to more X), potentially with a link to the documentation for further information is highly preferred.*

*If using boto3 and the AWS SDK, make sure to use the appropriate try: finally: logic on the waiter.*

In [ ]:
# Example logic for waiter
client = boto3.client('sagemaker')

client.create_training_job(**create_training_params)

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)

try:
    client.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
finally:
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print("Training job ended with status: " + status)
    if status == 'Failed':
        message = client.describe_training_job(TrainingJobName=job_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Training job failed')

---

## Host

*This section mainly includes the transaction aspects of hosting your algorithm with SageMaker.  Notes below may be handled to a greater or lesser extent depending on whether hosting is done using boto3 and the AWS SDK or the Python SDK.*

*The algorithm needs to host using the right container by region, similar to training above.*

*Unless there is an explicit reason not to, all hosting should be done on a single ml.m4.xlarge instance as they are eligible for the free tier.*

*Endpoints should always be named with a prefix of "DEMO" so that they can be properly distinguished from production endpoints.*

*If using boto3 and the AWS SDK, make sure to use the appropriate try: finally: logic in the waiter.*

In [ ]:
# Example logic for waiter
endpoint_name = 'DEMO-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    client.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)
finally:
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Create endpoint ended with status: " + status)

    if status != 'InService':
        message = client.describe_endpoint(EndpointName=endpoint_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Endpoint creation did not succeed')

### Evaluate

*This section should included details on how to serialize data in preparation for invoking the endpoint, deserializing the response, and understanding the model's outputs.*

---

## Extensions

*This section can be a simple paragraph on what could be done to improve upon or extend the existing example.  Examples would be to spend more time tuning hyperparameters, scale to a larger dataset, go through the next example notebook in a sequence, etc.*

### (Optional) Clean-up

*At the end of the example notebook make sure to have a cell that deletes the endpoint(s) you created.  Optionally, you may delete the S3 data, models, endpoints, or any other artifacts created during the example.*

_Prior to publishing the notebook, we typically clear all cell outputs using Cell -> All Output -> Clear in the Jupyter menu above._

*Confirm that the notebook runs end-to-end in a SageMaker Notebook Instance in multiple regions.  Review with your SDK and PM in advance if you have any questions.  Otherwise, submit a pull request to the [GitHub repo](https://github.com/awslabs/amazon-sagemaker-examples).*

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)